**PreRequisites:**
1. Install the following in your dev environment:<br>
    a. google-cloud-bigquery: pip.exe install google-cloud-bigquery<br>
    b. db-types: pip install db-dtypes<br>
2. Install gcloud CLI <br>
    a. Install directions (with download link): https://cloud.google.com/sdk/docs/install<br>
    > i. pay attention to where it installs!<br>
    > ii. It says to leave all the shortcut, open terminal options checked. I received errors when it ran "gcloud info --run-diagnostics" and I ignored them for now...<br>
    
    b. Add this to your PATH environmental variables (for me this was C:\Users\vt_be\AppData\Local\Google\Cloud SDK\google-cloud-sdk)<br>
    c. reboot!<br>
    d. open git bash, switch to dev environment<br>
    > i. "gcloud info --run-diagnostics" now ran without issue<br>
    ii. add authentication (this opens browser to connect your google account):  gcloud auth application-default login<br>
    
    e. I also needed to set up a Big Query Project: mostly followed https://cloud.google.com/bigquery/docs/sandbox<br>
    > i. I didn't see the stuff mentioned in #3 but otherwise worked<br>
    > ii. Note that when you create the project, an id is generated that is project name - #### (for me BootCamp-Weather:  bootcamp-weather-400118<br>
    
    f. Add the project to default - back to gitbash: gcloud auth application-default set-quota-project <project-id><br>
    g. In the downloaded notebook, add the project id to the client = bigquery.Client("project-id") in the first cell<br>
    

**Credit:**
* Big Query calls adapted from https://www.kaggle.com/code/crained/noaa-dataset-with-google-bigquery
* SQL calls adapted from GitHub BigQuery documentation: https://github.com/googleapis/python-bigquery

In [1]:
# My project name (don't think can be shared across people) is stored in a config.py file as "google_project"
# Since this is unique to user, I added config.py to the gitignore. You must create your own config.py file with project name
from config import google_project
# bigquery and pandas work well together for dataframes!
import pandas as pd
import os
# Follow the prerequisite instructions to get bigquery going
from google.cloud import bigquery
# Create a "Client" object reference a google project for which your system has been authenticated
client = bigquery.Client(google_project)

## Query for the Stations static info ##

In [2]:
# Station IDs (usaf) remained unique over time, but names changed. 
# This query will sort by begin date descending so we can use the remove duplicates function to keep the latest name

QUERY = (
    'SELECT usaf, name, state, lat, lon '
    'FROM `bigquery-public-data.noaa_gsod.stations` '
    'WHERE country = "US" AND state <> "None" '
    'ORDER BY begin DESC'
    )
# API request
stations_result = client.query(QUERY)  

# Waits for query to finish
stations_data = stations_result.result()  

# Put the last query into a dataframe
stations_data_df = stations_data.to_dataframe()

# Remove duplicate stations by usaf
stations_data_df = stations_data_df.drop_duplicates("usaf")

# and export
stations_data_df.to_json("GEOJSON_data/Stations.json", orient="records")
stations_data_df.to_csv("GEOJSON_data/Stations.csv")

print(len(stations_data_df))
stations_data_df.head()

3790


,usaf,name,state,lat,lon
0,720511,BRITTON MUNI,SD,45.815,-97.743
1,723062,PINEY ISLAND,NC,35.020,-76.460
2,A00030,CONNELLSVILLE AIRPORT,PA,39.959,-79.657
3,720844,SPANISH PEAKS,CO,37.697,-104.785
4,724916,MARINA MUNI,CA,36.682,-121.762


## Query for the statistics data ##
### Query for temperature stats (similar filtering) ###

In [3]:
aggregate_query = (
    'SELECT s.usaf, '
    'MIN(g.min) AS min_temp, '
    'AVG(g.temp) AS mean_temp, '
    'MAX(g.max) AS max_temp, '
    'FROM `bigquery-public-data.noaa_gsod.gsod2022` AS g '
    'INNER JOIN `bigquery-public-data.noaa_gsod.stations` AS s ON g.stn = s.usaf '
    'WHERE s.country = "US" AND s.state <> "None" '
    # This line below removes the 'not a reading' so we can run stats on those columns
    'AND g.min <> 9999.9 AND g.max <> 9999.9 '
    'GROUP BY s.usaf '
    )

# API request
station_temp_result = client.query(aggregate_query)  

# Waits for query to finish
station_temp_data = station_temp_result.result()  

# Put the last query into a dataframe
station_temp_df = station_temp_result.to_dataframe()

# # and export
# state_temp_station.to_json("GEOJSON_data/Station_temp_sample.json", orient="records")
# state_temp_station.to_csv("GEOJSON_data/Station_temp_sample.csv")

print(len(station_temp_df))
station_temp_df

2522


,usaf,min_temp,mean_temp,max_temp
0,700638,10.0,41.445956,70.0
1,702040,-5.1,30.590909,55.9
2,702490,-27.4,32.636119,77.0
3,702606,-7.6,24.352830,55.4
4,702628,-24.0,34.463977,81.0
...,...,...,...,...
2517,723408,15.8,54.384444,98.6
2518,720641,12.2,38.080000,62.6
2519,702084,35.1,51.180000,73.0
2520,701190,53.6,55.800000,57.2


### Query for total snow in a year ###

In [4]:
# Perform a query 
aggregate_query = (
    'SELECT s.usaf, '
    'SUM(g.sndp) AS total_snow '
    'FROM `bigquery-public-data.noaa_gsod.gsod2022` AS g '
    'INNER JOIN `bigquery-public-data.noaa_gsod.stations` AS s ON g.stn = s.usaf '
    'WHERE s.country = "US" AND s.state <> "None" '
    'AND g.sndp <> 999.9 '
    'GROUP BY s.usaf '
    'ORDER BY total_snow DESC')
station_snow_result = client.query(aggregate_query)  # API request
station_snow_data = station_snow_result.result()  # Waits for query to finish

# Put the last query into a dataframe
station_snow_df = station_snow_result.to_dataframe()

station_snow_df

,usaf,total_snow
0,702606,14721.2
1,702650,8622.6
2,702490,7887.4
3,702615,6871.8
4,701740,5199.6
...,...,...
276,722630,1.2
277,722280,1.2
278,722680,1.2
279,723320,1.2


### Query for total precipitation in a year ###

In [5]:
# Perform a query 
aggregate_query = (
    'SELECT s.usaf, '
    'SUM(g.prcp) AS total_precipitation '
    'FROM `bigquery-public-data.noaa_gsod.gsod2022` AS g '
    'INNER JOIN `bigquery-public-data.noaa_gsod.stations` AS s ON g.stn = s.usaf '
    'WHERE s.country = "US" AND s.state <> "None" '
    'AND g.prcp <> 99.9 '
    'GROUP BY s.usaf')
station_prcp_result = client.query(aggregate_query)  # API request
station_prcp_data = station_prcp_result.result()  # Waits for query to finish

# Put the last query into a dataframe
station_prcp_df = station_prcp_result.to_dataframe()

station_prcp_df

,usaf,total_precipitation
0,700860,12400.22
1,702040,8000.06
2,702490,10613.66
3,702606,8199.18
4,702607,2234.06
...,...,...
2518,A51256,17998.20
2519,997732,0.00
2520,723897,0.00
2521,998166,0.00


### Query for count of days with tornadoes in a year ###
SchemaField('tornado_funnel_cloud', 'STRING', 'NULLABLE', None, 'Indicators (1 = yes, 0 = no/not reported) for the occurrence during the day', (), None)

In [6]:
# Perform a query 
# Include month and day so can remove duplicate "yes" on the same day
QUERY = (
    'SELECT s.usaf, g.mo, g.da, '
    'g.tornado_funnel_cloud '
    'FROM `bigquery-public-data.noaa_gsod.gsod2022` AS g '
    'INNER JOIN `bigquery-public-data.noaa_gsod.stations` AS s ON g.stn = s.usaf '
    'WHERE s.country = "US" AND s.state <> "None" '
    'AND tornado_funnel_cloud = "1" '
    )

station_tornado_result = client.query(QUERY)  # API request
station_tornado_data = station_tornado_result.result()  # Waits for query to finish

# Put the last query into a dataframe
station_tornado_df = station_tornado_result.to_dataframe()

# Remove duplicate station day reports
station_tornado_nodup = station_tornado_df.drop_duplicates()
# Drop month and day
station_tornado_nodup = station_tornado_nodup[["usaf", "tornado_funnel_cloud"]]
# Count how many in the year for each station
station_tornado_count = station_tornado_nodup.groupby("usaf").count()

print(len(station_tornado_count))
station_tornado_count.head()

33


,tornado_funnel_cloud
usaf,
720381,1
722010,4
722015,8
722020,2
722030,1


### Query for count of days with hail in a year ###
SchemaField('hail', 'STRING', 'NULLABLE', None, 'Indicators (1 = yes, 0 = no/not reported) for the occurrence during the day', (), None)

In [7]:
# Perform a query 
# Include month and day so can remove duplicate "yes" on the same day
QUERY = (
    'SELECT s.usaf, g.mo, g.da, '
    'g.hail '
    'FROM `bigquery-public-data.noaa_gsod.gsod2022` AS g '
    'INNER JOIN `bigquery-public-data.noaa_gsod.stations` AS s ON g.stn = s.usaf '
    'WHERE s.country = "US" AND s.state <> "None" '
    'AND g.hail = "1" '
    )

station_hail_result = client.query(QUERY)  # API request
station_hail_data = station_hail_result.result()  # Waits for query to finish

# Put the last query into a dataframe
station_hail_df = station_hail_result.to_dataframe()

# Remove duplicate station day reports
station_hail_nodup = station_hail_df.drop_duplicates()
# Drop month and day
station_hail_nodup = station_hail_nodup[["usaf", "hail"]]
# Count how many in the year for each station
station_hail_count = station_hail_nodup.groupby("usaf").count()

print(len(station_hail_count))
station_hail_count.head()

113


,hail
usaf,
702650,2
702730,1
702910,1
704140,12
720619,1


## Merge all the data frames together ##

In [8]:
station_temp_merge = pd.merge(stations_data_df, station_temp_df, how ='inner', on = "usaf")

st_temp_snow_merge = pd.merge(station_temp_merge, station_snow_df, how ='outer', on = "usaf")

st_temp_snow_prcp = pd.merge(st_temp_snow_merge, station_prcp_df, how ='outer', on = "usaf")

station_stats_torn = pd.merge(st_temp_snow_prcp, station_tornado_count, how ='outer', on = "usaf")

station_all = pd.merge(station_stats_torn, station_hail_count, how ='outer', on = "usaf")

# and export
station_all.to_json("GEOJSON_data/stations_all.json", orient="records")
station_all.to_csv("GEOJSON_data/stations_all.csv")
station_all.to_json("GEOJSON_data/stations_all.js", orient="records")

station_all.head(-30)

,usaf,name,state,lat,lon,min_temp,mean_temp,max_temp,total_snow,total_precipitation,tornado_funnel_cloud,hail
0,720511,BRITTON MUNI,SD,45.815,-97.743,-23.8,41.988056,98.6,NaN,18198.18,NaN,NaN
1,723062,PINEY ISLAND,NC,35.020,-76.460,25.0,69.131609,99.0,NaN,4645.98,NaN,NaN
2,A00030,CONNELLSVILLE AIRPORT,PA,39.959,-79.657,-2.6,53.638226,91.8,NaN,17498.25,NaN,NaN
3,720844,SPANISH PEAKS,CO,37.697,-104.785,-14.3,54.005850,100.4,NaN,0.00,NaN,NaN
4,724916,MARINA MUNI,CA,36.682,-121.762,-11.2,42.305747,80.6,NaN,10798.92,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2488,749045,SEBRING RGNL,FL,27.456,-81.342,31.1,73.669670,98.6,NaN,3231.46,NaN,NaN
2489,726776,LEWISTOWN MUNICIPAL ARPT,MT,47.049,-109.458,-29.9,42.881044,100.9,85.0,17.28,NaN,NaN
2490,703160,COLD BAY AIRPORT,AK,55.221,-162.732,3.9,40.355068,69.1,NaN,140.96,NaN,NaN
2491,722340,KEY FIELD AIRPORT,MS,32.335,-88.744,12.9,65.196986,104.0,NaN,54.58,NaN,NaN
